## Appendix-7: Deep neural network with convolution layer (CNN):

This method uses a deep neural network architecture with convolutional layers to classify the CIFAR-10 dataset. The network consisted of several convolutional layers, each followed by a ReLU activation function and a max-pooling layer for downsampling, and several fully connected layers for classification. 

We used PyTorch to implement the model and trained it using the cross-entropy loss function and the stochastic gradient descent (SGD) optimizer.

In [1]:
# import library dependencies
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

#### Import Data

In [2]:
ROOT_PATH='../'

In [3]:
BATCH_SIZE=16

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
train_dataset = CIFAR10(root=ROOT_PATH, download=True, train=True, transform=transform)
eval_dataset = CIFAR10(root=ROOT_PATH, train=False, transform=transform)

Files already downloaded and verified


#### Preprocess Data

In [6]:
train_data_loader = DataLoader(dataset=train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
eval_data_loader = DataLoader(dataset=eval_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=False)

#### Define model and train

In [7]:
# define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64 * 8 * 8)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

In [8]:
# create an instance of the CNN
net = CNN()

In [9]:
# define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [10]:
num_epochs = 50

In [11]:
# train the CNN
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_data_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.816
[2,  2000] loss: 1.299
[3,  2000] loss: 1.097
[4,  2000] loss: 0.949
[5,  2000] loss: 0.833
[6,  2000] loss: 0.729
[7,  2000] loss: 0.629
[8,  2000] loss: 0.530
[9,  2000] loss: 0.433
[10,  2000] loss: 0.337
[11,  2000] loss: 0.249
[12,  2000] loss: 0.176
[13,  2000] loss: 0.111
[14,  2000] loss: 0.073
[15,  2000] loss: 0.044
[16,  2000] loss: 0.021
[17,  2000] loss: 0.014
[18,  2000] loss: 0.007
[19,  2000] loss: 0.003
[20,  2000] loss: 0.002
[21,  2000] loss: 0.002
[22,  2000] loss: 0.005
[23,  2000] loss: 0.002
[24,  2000] loss: 0.001
[25,  2000] loss: 0.001
[26,  2000] loss: 0.001
[27,  2000] loss: 0.001
[28,  2000] loss: 0.001
[29,  2000] loss: 0.001
[30,  2000] loss: 0.001
[31,  2000] loss: 0.001
[32,  2000] loss: 0.001
[33,  2000] loss: 0.001
[34,  2000] loss: 0.001
[35,  2000] loss: 0.000
[36,  2000] loss: 0.000
[37,  2000] loss: 0.000
[38,  2000] loss: 0.000
[39,  2000] loss: 0.000
[40,  2000] loss: 0.000
[41,  2000] loss: 0.000
[42,  2000] loss: 0.000
[

#### Save and load model

In [12]:
# save the trained model
torch.save(net, 'cnn.pth')

In [13]:
# load the saved model
net = torch.load('cnn.pth')

#### Evaluate the model

In [14]:
# evaluate the CNN
net.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in eval_data_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 73 %


In [15]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [16]:
# calculate class-wise accuracy
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in eval_data_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(BATCH_SIZE):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        class_names[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 79 %
Accuracy of   car : 83 %
Accuracy of  bird : 63 %
Accuracy of   cat : 55 %
Accuracy of  deer : 69 %
Accuracy of   dog : 63 %
Accuracy of  frog : 82 %
Accuracy of horse : 78 %
Accuracy of  ship : 83 %
Accuracy of truck : 81 %


In [18]:
TP = 0
FP = 0
TN = 0
FN = 0

with torch.no_grad():
    for data in eval_data_loader:
        images, labels = data
        images = images
        labels = labels
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        for i in range(len(labels)):
            if predicted[i] == labels[i]:
                if predicted[i] == 1:
                    TP += 1
                else:
                    TN += 1
            else:
                if predicted[i] == 1:
                    FP += 1
                else:
                    FN += 1

accuracy = 100 * (TP + TN) / (TP + TN + FP + FN)
precision = 100 * TP / (TP + FP)
recall = 100 * TP / (TP + FN)
f1_score = 2 * precision * recall / (precision + recall)

print('Accuracy: %.2f %%' % (accuracy))
print('Precision: %.2f %%' % (precision))
print('Recall: %.2f %%' % (recall))
print('F1 Score: %.2f %%' % (f1_score))

Accuracy: 73.97 %
Precision: 83.42 %
Recall: 25.52 %
F1 Score: 39.08 %
